In [1]:
import pandas as pd
import pyarrow.parquet as pa
import pyarrow.dataset as ds

from azure.ai.ml import MLClient
from azure.identity import ClientSecretCredential
from azure.mgmt.storage import StorageManagementClient
from azure.ai.ml import Input, Output, command
from azureml.core import Workspace
from azureml.fsspec import AzureMachineLearningFileSystem
from tqdm.autonotebook import tqdm
import sys, os
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Environment

/tmp/ipykernel_50005/2830835052.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# At the time I developed this code I couldn't find any SDK V2 version to do this

ws = Workspace.from_config()
kv = ws.get_default_keyvault()

In [3]:
sp_id = kv.get_secret('SP-APPID')
sp_secret = kv.get_secret('SP-SECRET')
tenant_id = kv.get_secret('TENANT-ID')

In [4]:

credentials = ClientSecretCredential(tenant_id,sp_id,sp_secret)
ml_client = MLClient(credentials, ws.subscription_id,ws.resource_group, ws._workspace_name)

# Create new env

In [5]:
dependencies_dir = "./env"
os.makedirs(dependencies_dir, exist_ok=True)
def create_environment(custom_env_name, dependencies_dir):
    file_path = os.path.join (dependencies_dir,f"{custom_env_name}.yml")
    print(file_path)

    assert os.path.isfile(file_path)
    job_env = Environment(
        name=custom_env_name,
        description="mlflow sklearn environment",
        conda_file=os.path.join(dependencies_dir, f"{custom_env_name}.yml"),
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
        )
    job_env = ml_client.environments.create_or_update(job_env)
    return job_env
env2 = create_environment("env3", "env")

env/env3.yml


In [ ]:
for env in ml_client.environments.list():
    print(env.name, env.latest_version)

In [7]:
env = ml_client.environments.get("env3","2")

In [9]:
cpu_compute_target_name = "CLUSTER-001"
cpu_compute_target = ml_client.compute.get(cpu_compute_target_name)

# Data

## datastore

In [10]:
for ds in ml_client.datastores.list():
    print(ds.name)

azureml_globaldatasets
workspaceworkingdirectory
workspacefilestore
workspaceartifactstore
workspaceblobstore


In [11]:
blb_store = ml_client.datastores.get("workspaceblobstore")

## Upload to azure

### parquet files

In [12]:
data_path = "data"
file_name = "promotion"
ship_df = pd.read_csv(f"{data_path}{file_name}.csv", dtype=str)
ship_df.to_parquet(f"{data_path}{file_name}.parquet",partition_cols=["Item.[L6]"],index=False)

ship_data = Data(
    name=f"{file_name}_5cat_2023_w38",
    description=f"{file_name} for 5 partitions of data and week 2023-W38",
    path= f"./{data_path}{file_name}.parquet",
    type=AssetTypes.URI_FOLDER,
)
ml_client.data.create_or_update(ship_data)

Uploading price_promo_compass.parquet (5.49 MBs): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5487092/5487092 [00:00<00:00, 12899661.92it/s]


Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'price_promo_compass_5cat_2023_w38', 'description': 'price_promo_compass for 5 categories 050_026_233_227_239 for week 2023-W38', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/4af96d0b-502f-44fc-a9b8-18cef735f3f6/resourceGroups/KHAZ-EUS2-RG-USO9-001-EXPD/providers/Microsoft.MachineLearningServices/workspaces/ML-EXPD-CI5220-USO9-001/data/price_promo_compass_5cat_2023_w38/versions/3', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/matt-uso9-expd-ci03/code/Users/mohammad.najarian/azureml_practice', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fe18558cbb0>, 'serialize': <msrest.serialization.Serializer object at 0x7fe18558c5b0>, 'version': '3', 'latest_version': None, 'path': 'azureml://subscriptions/4af96d0b-502f-

### CSV files

In [13]:
data_path = "data"
file_name = "prod_hierarchy"
ship_df = pd.read_csv(f"{data_path}{file_name}.csv", dtype=str)
ship_data = Data(
    name=f"{file_name}_5part_2023_w38",
    description=f"{file_name} for 5 partitions of data and week 2023-W38",
    path= f"./{data_path}{file_name}.csv",
    type=AssetTypes.URI_FILE,
)
ml_client.data.create_or_update(ship_data)

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'prod_hierarchy_5cat_2023_w38', 'description': 'prod_hierarchy for 5 categories 050_026_233_227_239 for week 2023-W38', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/4af96d0b-502f-44fc-a9b8-18cef735f3f6/resourceGroups/KHAZ-EUS2-RG-USO9-001-EXPD/providers/Microsoft.MachineLearningServices/workspaces/ML-EXPD-CI5220-USO9-001/data/prod_hierarchy_5cat_2023_w38/versions/3', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/matt-uso9-expd-ci03/code/Users/mohammad.najarian/azureml_practice', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fe1873c7280>, 'serialize': <msrest.serialization.Serializer object at 0x7fe1873c6770>, 'version': '3', 'latest_version': None, 'path': 'azureml://subscriptions/4af96d0b-502f-44fc-a9b8-18cef73

In [14]:
try:
    compass_data = ml_client.data.get("promotion", version=1)
    print("data exists")
except:
    compass_data = Data(
        name="promotion",
        description="the promotion data",
        path="./data/promotion_2022_W49.parquet/",
        type=AssetTypes.URI_FOLDER
    )
    ml_client.data.create_or_update(compass_data)
    print("data diden't exist and we uploaded the data from local")

data exists


### get data objects

In [ ]:
for dd in ml_client.data.list():
    print(dd.name, dd.latest_version)


In [16]:
gen_week = "2023_w38"
tag = '5promo'
price_promo_data = ml_client.data.get(f"promotion_{tag}_{gen_week}", version=1)
shipment_data = ml_client.data.get(f"shipment_{tag}_{gen_week}", version=1)
prod_hierarchy_data = ml_client.data.get(f"prod_hierarchy_{tag}_{gen_week}", version=1)
holidays_data = ml_client.data.get(f"holidays_{tag}_{gen_week}", version=1)

## Construct connection string

In [ ]:
storage_client = StorageManagementClient(credentials, ws.subscription_id)
keys = storage_client.storage_accounts.list_keys(ws.resource_group, blb_store.account_name)
print(f"Primary key for storage account: {keys.keys[0].value}")
conn_string = f"DefaultEndpointsProtocol=https;EndpointSuffix=core.windows.net;AccountName={blb_store.account_name};AccountKey={keys.keys[0].value}"



In [19]:
aml_fs = AzureMachineLearningFileSystem(compass_data.path)
files = aml_fs.glob()

## Read data


### Using AzureMachineLearningFileSystem (works)

In [23]:
import pandas as pd

# define the URI - update <> placeholders
def read_parqut_from_azure(path, partition = None):


    # create the filesystem
    fs = AzureMachineLearningFileSystem(path)

    # append csv files in folder to a list
    dflist = []
    if partition is not None:
        path = path + partition
    for path in tqdm (fs.ls()):
        with fs.open(path) as f:
            dflist.append(pd.read_parquet(f))

    # # concatenate data frames
    df = pd.concat(dflist)
    return df
df = read_parqut_from_azure(price_promo_data.path, 'Item.[L6]=227')

  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0/5 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  9.15it/s]


In [24]:
sys.getsizeof(df) / 2**20

253.72820949554443

# Create Components

In [25]:
from azure.ai.ml import Input, Output, command

input types:  'uri_folder', 'uri_file', 'mltable', 'mlflow_model', 'custom_model', 'integer', 'number', 'string', and 'boolean'. Defaults to 'uri_folder'.

In [26]:
def command_gen(code_file=None, input=None, output=None):
    """A function to generate commands automatically

    Args:
        code_file (string, optional): file name. Defaults to None.
        input (dict, optional): the inputs object. Defaults to None.
        output (dict, optional): the outputs object. Defaults to None.

    Returns:
        string: the command
    """
    cmd = f"""python {code_file} """
    if input is not None:
        for k in input.keys():
            cmd += """--{0} ${1}{0}{2} """.format(k,"{{inputs.", "}}  ")
    if output is not None:
        for k in output.keys():
            cmd += """--{0} ${1}{0}{2} """.format(k,"{{outputs.", "}}  ")
    return cmd


In [27]:

env.version

'2'

In [53]:
inputs={
    "shipment" : Input(type='uri_folder'),
    "promotion": Input(type="uri_folder"),
    "holidays":Input(type='uri_file'),
    'product_hierarchy':Input(type='uri_file'),
    'category':Input(type="number"),
    'generation_week':Input(type='string'),
    'period':Input(type='string')
}

outputs={
    'train_data':Output(type="uri_file", mode="rw_mount"),
    'pred_data':Output(type="uri_file", mode="rw_mount"),
}
data_prep = command(
    name="data_prep_hierarchical",
    display_name="data prep for hierarchical forecasting",
    description="read category data for hierarchical forecasting",
    inputs=inputs,
    outputs=outputs,
    code="./src",
    command=command_gen("prepare_data_v2.py", inputs,outputs),
    environment=f"{env.name}:{env.version}",
)
data_prep_component = ml_client.create_or_update(data_prep.component)

Uploading src (0.01 MBs): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9174/9174 [00:00<00:00, 83263.10it/s]


In [46]:
data_prep = ml_client.components.get("data_prep_hierarchical")

# Create pipeline

In [47]:
from azure.ai.ml import dsl


@dsl.pipeline(
    compute=cpu_compute_target if (cpu_compute_target) else "serverless",
    description="hierarchical data_perp-train pipeline",
)
def hierarchical_pipeline(
    shipment_path, promotion_path, holiday_path, prod_hierarchy_path, category, generation_week
):
    data_prep_job = data_prep(
        shipment=shipment_path,
        promotion=promotion_path,
        holidays=holiday_path,
        product_hierarchy=prod_hierarchy_path,
        category=category,
        generation_week=generation_week,
        period="W",
    )

    return {
        "pipeline_job_train_data": data_prep_job.outputs.train_data,
        "pipeline_job_test_data": data_prep_job.outputs.pred_data,
    }




@dsl.pipeline(
    compute=cpu_compute_target if (cpu_compute_target) else "serverless",
    description="hierarchical data_perp-train pipeline",
)
def hierarchical_pipeline(
    shipment_path, promotion_path, holiday_path, prod_hierarchy_path, category
):
    data_prep_job = data_prep(
        shipment=Input(type="uri_folder", path=shipment_path),
        promotion=Input(type="uri_folder", path=promotion_path),
        holidays=Input(type="uri_file", path=holiday_path),
        product_hierarchy=Input(type="uri_file", path=prod_hierarchy_path),
        category=category,
        generation_week="2023-W16",
        period="W",
    )

    return {
        "pipeline_job_train_data": data_prep_job.outputs.train_data,
        "pipeline_job_test_data": data_prep_job.outputs.pred_data,
    }

In [ ]:
read_parqut_from_azure(shipment_data.path,"Item.[L6]=227")

In [48]:
gen_week = "2023_w38"
tag = '5cat'
price_promo_data = ml_client.data.get(f"promotion{tag}_{gen_week}", version=1)
shipment_data = ml_client.data.get(f"shipment_{tag}_{gen_week}", version=1)
prod_hierarchy_data = ml_client.data.get(f"prod_hierarchy_{tag}_{gen_week}", version=1)
holidays_data = ml_client.data.get(f"holidays_{tag}_{gen_week}", version=1)

In [54]:
pipeline = hierarchical_pipeline(
    shipment_path=Input(type="uri_folder", path=shipment_data.path),
    promotion_path=Input(type="uri_folder", path=price_promo_data.path),
    prod_hierarchy_path=Input(type="uri_file", path=prod_hierarchy_data.path),
    holiday_path=Input(type="uri_file", path=holidays_data.path),
    category=26,
    generation_week=gen_week,
)
pipeline_job = ml_client.jobs.create_or_update(
    pipeline, experiment_name="hierarchical_test_data_read"
)
ml_client.jobs.stream(pipeline_job.name)


RunId: sharp_lock_qcmm60phsn
Web View: https://ml.azure.com/runs/sharp_lock_qcmm60phsn?wsid=/subscriptions/4af96d0b-502f-44fc-a9b8-18cef735f3f6/resourcegroups/KHAZ-EUS2-RG-USO9-001-EXPD/workspaces/ML-EXPD-CI5220-USO9-001

Streaming logs/azureml/executionlogs.txt

[2023-10-21 20:50:00Z] Submitting 1 runs, first five are: 5d825935:8ca997b5-93e8-4dc7-937c-860225427b0a
[2023-10-21 20:55:34Z] Completing processing run id 8ca997b5-93e8-4dc7-937c-860225427b0a.

Execution Summary
RunId: sharp_lock_qcmm60phsn
Web View: https://ml.azure.com/runs/sharp_lock_qcmm60phsn?wsid=/subscriptions/4af96d0b-502f-44fc-a9b8-18cef735f3f6/resourcegroups/KHAZ-EUS2-RG-USO9-001-EXPD/workspaces/ML-EXPD-CI5220-USO9-001

